### PCA example
PCA to najbardziej popularny algorytm redukcji wymiarów. W ogólnym skrócie polega on na rzutowaniu danych do przestrzeni o mniejszej liczbie wymiarów tak, aby jak najlepiej zachować strukturę danych. 

Poniżej przykład wykorzystania PCA do redukcji wymiarów na podstawie zbioru o Bankach na podstawie raportów dostępnych na stronie www.PRnews.pl na dzień 2019-10-15.

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

seed = 2019

#### Import danych

In [ ]:
df = pd.read_excel('./data/data.xlsx', sheet_name='dataframe')
df.sort_values(by='2019Q2_aktywa', ascending=False).head()

#### Preprocessing zbioru: standaryzacja i obsłużenie pustych wartości.   

Przed wykonaniem PCA należy dokonać standaryzacji zbioru danych (średnia = 0, wariancja = 1) ze względu na różną skalę jaka może występować w zmiennych - np. waga i dochody. Dane są następnie normalizowane (skalowane do przedziału [0,1]).
<br/>

Dodatkowo implementacja PCA w pakiecie scikit-learn nie obsługuje pustych wartości, dlatego też należy odpowiednio je obsłużyć. 
<br/>

**Uwaga**: W tym przypadku zapominamy, że mamy jakikolwiek zbiór do walidacji\testowania modelu (jak ma to miejsce dla uczenia nadzorowanego) i wykonujemy funkcję fit_transform() na całym zbiorze (który może być traktowany jako zbiór treningowy). Dla uczenia nadzorowanego wykonywalibyśmy też krok następny, czyli funkcję transform() na zbiorze walidacyjnym\testowym.

In [ ]:
def apply_scalers(df, columns_to_exclude=None):
    if columns_to_exclude:
        exclude_filter = ~df.columns.isin(columns_to_exclude) 
    else:
        exclude_filter = ~df.columns.isin([]) 
    for column in df.iloc[:, exclude_filter].columns:
        df[column] = df[column].astype(float)

    df.loc[:, exclude_filter] = StandardScaler().fit_transform(df.loc[:, exclude_filter])
    return df

df = apply_scalers(df, columns_to_exclude=['Nazwa'])
df.sort_values(by='2019Q2_aktywa', ascending=False).head()

#### PCA

W celu wykonania analizy głównych składowych można wykorzystać funkcję *PCA* z pakietu *sklearn.decomposition*. Najważniejsze parametry funkcji: 

* *n_components* - liczba *n* czynników w nowej przestrzeni 
* *svd_solver* - typ dekompozycji macierzy. Dostępne wartości: *auto, full, arpack, randomized*

Więcej o parametrach: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
# kolumny do wykluczenia (te na których nie chcemy PCA)
exclude_filter = ~df.columns.isin(['Nazwa']) 
# liczba głównych składowych
pca = PCA(n_components = 3)
# przeliczenie
principal_components = pca.fit_transform(df.loc[:, exclude_filter])

In [ ]:
principal_df = pd.DataFrame(data = principal_components, 
                           columns = ['principal component 1',
                                      'principal component 2',
                                      'principal component 3']
                           )

principal_df['Nazwa'] = df['Nazwa']
principal_df

In [ ]:
!pip install chart-studio

In [ ]:
import pandas as pd
import seaborn as sns

from chart_studio import plotly as py
#import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
trace0 = go.Scatter(
    x = principal_df['principal component 1'],
    y = principal_df['principal component 2'],
    text=principal_df['Nazwa'],
    textposition="top center",
    name = 'Piony',
    mode = 'markers+text',
    marker = dict(
        size = 10,
        color = 'rgb(228,26,28)',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)

data = [trace0]

layout = dict(title = 'Podobieństwo Banków na podstawie PCA',
              yaxis = dict(zeroline = False, title ='PC2 (principal component 2)'),
              xaxis = dict(zeroline = False, title ='PC1 (principal component 1)')
             )

fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')

#### PCA - jak dobrać liczbę komponentów? 

Pierwszym ze sposób jest wizualizacja skumulowanej wartości wariancji w zależności od liczby komponentów. Z wykresu można  odczytać, że 4 pierwszych czynników może wyjaśnić ponad 95% całkowitej wariancji.  

In [ ]:
import numpy as np

pca = PCA().fit(df.loc[:, exclude_filter])
plt.figure(figsize=(10,6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

##### Wykorzystanie parametru svd_solver.

Jeżeli **0 < *n_components* < 1** oraz ***svd_solver = 'full'*** to funkcja PCA wybiera liczbę komponentów, tak aby wielkość wariancji, którą należy wyjaśnić, była większa niż procent określony przez *n_components*.

In [ ]:
pca = PCA(svd_solver='full', n_components=0.95)

principal_components = pca.fit_transform(df.loc[:, exclude_filter])
principal_df = pd.DataFrame(data=principal_components)
principal_df.head()